In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from decoder import VAE_AttentionBlock, VAE_ResidualBlock



In [ ]:
class VAE_Encoder(nn.Sequential): # sequence of modules to reduce the dimensionality of the input
    def __init__(self):
        super().__init__(
            nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1),

            VAE_ResidualBlock(128, 128), # maybe to increase the depth of the network
            VAE_ResidualBlock(128, 128), # maybe to increase the number of parameters
            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=0),  # height and width are halved

            VAE_ResidualBlock(128, 256),
            VAE_ResidualBlock(256, 256),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=0),  # height and width are halved

            VAE_ResidualBlock(256, 512),
            VAE_ResidualBlock(512, 512),
            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=0),  # height and width are halved

            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            VAE_AttentionBlock(512),

            VAE_ResidualBlock(512, 512),
            # nn.GroupNorm(32, 512), # 32 is the number of groups(BatchNorm2d(512) is the same)
            nn.BatchNorm2d(512),

            nn.SiLU(), # nn.ReLU(),

            nn.Conv2d(512, 8, kernel_size=3, padding=1), 
            nn.Conv2d(8, 8, kernel_size=1, padding=0), # linear transformation to reduce the dimensionality of the input
            
        )

    def forward(self, x:torch.Tensor, noice:torch.Tensor)-> torch.Tensor:
        # x: (batch_size, 3, 512, 512)
        # noise: (batch_size, 3, 512, 512)
        for modules in self:
            print(modules)
            if isinstance(modules, VAE_AttentionBlock):
                x = modules(x, noice)
            else:
                x = modules(x)

